In [1]:
# [RDKitによる3次元構造の生成 | 化学の新しいカタチ](https://future-chem.com/rdkit-3dstruct/)
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
import sys, py3Dmol
import pandas as pd

RDKit WARNING: [13:32:11] Enabling RDKit 2019.09.2 jupyter extensions


In [2]:
print(sys.version)
print(rdBase.rdkitVersion)

3.7.5 (default, Oct 25 2019, 10:52:18) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
2019.09.2


In [3]:
# The Platinum Dataset is downloaded from below link
# [Servers & Datasets](https://comp3d.univie.ac.at/servers-datasets/#c523270)
# read molecule
suppl = Chem.SDMolSupplier('./sdf/platinum_dataset_2017_01.sdf', removeHs=False)
mols = [x for x in suppl if x is not None]
len(mols)

4548

In [5]:
# re-building molecules by exchange SMILES
smiles = [Chem.MolToSmiles(m) for m in mols]
mols_from_sm = [Chem.MolFromSmiles(sm) for sm in smiles]
mols_from_sm = mols_from_sm[0:2]  # mols_from_sm is too long so adjusted

In [9]:
### create 3D object for each way

def DG(mols):
    DG_uff = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, useBasicKnowledge=False, useExpTorsionAnglePrefs=False)
        if AllChem.UFFHasAllMoleculeParams(mh):
            AllChem.UFFOptimizeMolecule(mh)
            DG_uff.append(mh)
    return DG_uff

def ETDG(mols):
    ETDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, AllChem.ETDG())
        ETDG_mols.append(mh)
    return ETDG_mols

def ETKDG(mols, version=1):
    ETKDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        if version == 1:
            p = AllChem.ETKDG()
        elif version == 2:
            p = AllChem.ETKDGv2()
        else:
            print('invalid input')
        AllChem.EmbedMolecule(mh, p)
        ETKDG_mols.append(mh)
    return ETKDG_mols

# DG_m = DG(mols_from_sm)
# ETDG_m = ETDG(mols_from_sm)
# ETKDGv1_m = ETKDG(mols_from_sm)
# ETKDGv2_m = ETKDG(mols_from_sm, 2)

# # get crystal structure and RMDS
# RMSD = []
# for mol, dg, etdg, etkdg1, etkdg2 in zip(mols, DG_m, ETDG_m, ETKDGv1_m, ETKDGv2_m):
#     DG_rms = AllChem.GetBestRMS(mol, dg)
#     ETDG_rms = AllChem.GetBestRMS(mol, etdg)
#     ETKDG1_rms = AllChem.GetBestRMS(mol, etkdg1)
#     ETKDG2_rms = AllChem.GetBestRMS(mol, etkdg2)
#     RMSD.append((DG_rms, ETDG_rms, ETKDG1_rms, ETKDG2_rms))
# RMSD = pd.DataFrame(RMSD, columns=['DG+UFF', 'ETDG', 'ETKDG', 'ETKDGv2'])
# RMSD.describe().round(2)

In [7]:
DG_m = DG(mols_from_sm)
ETDG_m = ETDG(mols_from_sm)
ETKDGv1_m = ETKDG(mols_from_sm)
ETKDGv2_m = ETKDG(mols_from_sm, 2)

In [10]:
# get crystal structure and RMDS
RMSD = []
for mol, dg, etdg, etkdg1, etkdg2 in zip(mols, DG_m, ETDG_m, ETKDGv1_m, ETKDGv2_m):
    DG_rms = AllChem.GetBestRMS(mol, dg)
    ETDG_rms = AllChem.GetBestRMS(mol, etdg)
    ETKDG1_rms = AllChem.GetBestRMS(mol, etkdg1)
    ETKDG2_rms = AllChem.GetBestRMS(mol, etkdg2)
    RMSD.append((DG_rms, ETDG_rms, ETKDG1_rms, ETKDG2_rms))
RMSD = pd.DataFrame(RMSD, columns=['DG+UFF', 'ETDG', 'ETKDG', 'ETKDGv2'])
RMSD.describe().round(2)

,DG+UFF,ETDG,ETKDG,ETKDGv2
count,2.00,2.00,2.00,2.00
mean,2.75,2.62,3.08,3.10
std,0.47,0.36,0.80,0.84
min,2.42,2.37,2.51,2.51
25%,2.58,2.50,2.79,2.80
50%,2.75,2.62,3.08,3.10
75%,2.91,2.75,3.36,3.39
max,3.08,2.88,3.64,3.69


In [13]:
IPythonConsole.drawMol3D(mols[-1])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
v = py3Dmol.view(width=400, height=400)
mols_show = [DG_m[1], ETDG_m[1], ETKDGv1_m[1], ETKDGv2_m[1]]
for mol in mols_show:
    mb = Chem.MolToMolBlock(mol)
    v.addModel(mb, 'sdf')
v.setBackgroundColor('0xeeeeee')
v.setStyle({'stick': {}})
v.zoomTo()
v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
v = py3Dmol.view(width=600, height=400, linked=False, viewergrid=(2,2))
for m, i in zip(mols[0:1], [(0,0), (0,1), (1,0), (1,1)]):
    mb = Chem.MolToMolBlock(m)
    v.addModel(mb, 'sdf', viewer=i)
v.setBackgroundClolor('0xeeeeee')
v.setStyle({'stick': {}})
v.zoomTo()
v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol